# Movie Recommender 


[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/mahdi-ebrahimi-per/AI/blob/main/5-%20Recommender/Recommender.ipynb)


<br>
<br>
<br>


# Packages  

In [2]:
import numpy as np
import pandas as pd



# Load anad Config DataSet

In [3]:
r_cols = ["user_id", "movie_id", "rating"]   # rating columns
ratings = pd.read_csv("movies_data.txt", sep= '\t', names=r_cols, usecols=range(3))
ratings.head()

m_cols = ['movie_id', 'title'] # movie column
movies = pd.read_csv("movies_item.txt", sep="|", names=m_cols, usecols=range(2))
# movies.head()

ratings = pd.merge(movies, ratings)
ratings.head()


,movie_id,title,user_id,rating
0,1,Toy Story (1995),308,4
1,1,Toy Story (1995),287,5
2,1,Toy Story (1995),148,4
3,1,Toy Story (1995),280,4
4,1,Toy Story (1995),66,3


# Rating table per users
show all user (user_id) ratings in one row ,  if user doesn't see movie, the value of that will be NaN 

In [4]:
moviesRatings = ratings.pivot_table(index='user_id' , columns='title' , values='rating')

test_rating = moviesRatings['Toy Story (1995)']

moviesRatings.head()


title,'Til There Was You (1997),1-900 (1994),101 Dalmatians (1996),12 Angry Men (1957),187 (1997),2 Days in the Valley (1996),"20,000 Leagues Under the Sea (1954)",2001: A Space Odyssey (1968),3 Ninjas: High Noon At Mega Mountain (1998),"39 Steps, The (1935)",...,Wyatt Earp (1994),Yankee Zulu (1994),Year of the Horse (1997),You So Crazy (1994),Young Frankenstein (1974),Young Guns (1988),Young Guns II (1990),"Young Poisoner's Handbook, The (1995)",Zeus and Roxanne (1997),� k�ldum klaka (Cold Fever) (1994)
user_id,,,,,,,,,,,,,,,,,,,,,
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,2.0,5.0,NaN,NaN,3.0,4.0,NaN,NaN,...,NaN,NaN,NaN,NaN,5.0,3.0,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,2.0,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


# Find similar movies (First Algorithm)
    - check ratings of the movies that like user seen movie (similarity)

but this way not work good

In [5]:
StarWars_Rating = moviesRatings['Star Wars (1977)']
SimilarMoive = moviesRatings.corrwith(StarWars_Rating)
SimilarMoive.dropna()  # drop NaN values
SimilarMoive.sort_values(ascending=False)
SimilarMoive_DF =pd.DataFrame(SimilarMoive)
SimilarMoive_DF.head()


C:\Users\asus\AppData\Local\Programs\Python\Python39\lib\site-packages\numpy\lib\function_base.py:2634: RuntimeWarning: Degrees of freedom <= 0 for slice
  c = cov(x, y, rowvar, dtype=dtype)
C:\Users\asus\AppData\Local\Programs\Python\Python39\lib\site-packages\numpy\lib\function_base.py:2493: RuntimeWarning: divide by zero encountered in true_divide
  c *= np.true_divide(1, fact)


,0
title,
'Til There Was You (1997),0.872872
1-900 (1994),-0.645497
101 Dalmatians (1996),0.211132
12 Angry Men (1957),0.184289
187 (1997),0.027398


# Find similar movies (Secend Algorithm)
    - considering joined : similarity and ratings mean and ratings count
    
    - sort based on similarity


the point is this algorithm is for comparison of just one movie (our example is similar of Star wars) to recommend similar movies


In [6]:
moviesStats = ratings.groupby('title').agg({'rating' : [np.size, np.mean]})
moviesStats.head(10)

popularMovies = moviesStats["rating"]["size"] >= 100
moviesStats[popularMovies].sort_values([('rating', 'mean')], ascending=False)[:10]


df = moviesStats[popularMovies].join(pd.DataFrame(SimilarMoive, columns=['similarity']))
df.sort_values(['similarity'], ascending=False)

C:\Users\asus\AppData\Local\Programs\Python\Python39\lib\site-packages\pandas\core\reshape\merge.py:648: UserWarning: merging between different levels can give an unintended result (2 levels on the left,1 on the right)
  warnings.warn(msg, UserWarning)


,"(rating, size)","(rating, mean)",similarity
title,,,
Star Wars (1977),584,4.359589,1.000000
"Empire Strikes Back, The (1980)",368,4.206522,0.748353
Return of the Jedi (1983),507,4.007890,0.672556
Raiders of the Lost Ark (1981),420,4.252381,0.536117
Austin Powers: International Man of Mystery (1997),130,3.246154,0.377433
...,...,...,...
"Edge, The (1997)",113,3.539823,-0.127167
As Good As It Gets (1997),112,4.196429,-0.130466
Crash (1996),128,2.546875,-0.148507


# Find similar movies (third Algorithm)
    - The higher the score of a movie, the higher the prediction we can make

    
    - ?
    

the point is this algorithm is for comparison of multiple movies to recommend similar movies


In [9]:
# corrMatrix = moviesRatings.corr()
# corrMatrix.head()

corrMatrix = moviesRatings.corr(method="pearson", min_periods=100)
    # min periods -> هر چه تعداد رای به فیلم ها بیشتر باشد، اتکای بالاتری داریم، 
        # این مشخص میکند که حداقل چند رای برای فیلم باشد
corrMatrix.head()

title,'Til There Was You (1997),1-900 (1994),101 Dalmatians (1996),12 Angry Men (1957),187 (1997),2 Days in the Valley (1996),"20,000 Leagues Under the Sea (1954)",2001: A Space Odyssey (1968),3 Ninjas: High Noon At Mega Mountain (1998),"39 Steps, The (1935)",...,Wyatt Earp (1994),Yankee Zulu (1994),Year of the Horse (1997),You So Crazy (1994),Young Frankenstein (1974),Young Guns (1988),Young Guns II (1990),"Young Poisoner's Handbook, The (1995)",Zeus and Roxanne (1997),� k�ldum klaka (Cold Fever) (1994)
title,,,,,,,,,,,,,,,,,,,,,
'Til There Was You (1997),NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1-900 (1994),NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
101 Dalmatians (1996),NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
12 Angry Men (1957),NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
187 (1997),NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [12]:
userID = 0
myRatings = moviesRatings.loc[userID].dropna()
print(myRatings)

title
Empire Strikes Back, The (1980)    5.0
Gone with the Wind (1939)          1.0
Star Wars (1977)                   5.0
Name: 0, dtype: float64
